In [1]:
import keras
import tensorflow
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

Using TensorFlow backend.


In [2]:
# Change runtime to GPU and check if GPU is available or not 
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
# DATA - https://www.kaggle.com/c/digit-recognizer/data
# Kaggle 
! pip install kaggle
! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
# Or manually copy credentials in kaggle.json file 

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
!  echo '{"username":"shivamkotwalia","key":"599101619fc6e1fb0b0e30b2b827e8b1"}' > /root/.kaggle/kaggle.json

In [9]:
! cat /root/.kaggle/kaggle.json
! chmod 600 /root/.kaggle/kaggle.json

{"username":"shivamkotwalia","key":"599101619fc6e1fb0b0e30b2b827e8b1"}


In [8]:
# Downlaod data
! kaggle competitions download -c digit-recognizer

404 - Not Found


In [7]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

FileNotFoundError: ignored

In [0]:
print(df_train.shape)
df_train.head()

In [0]:
x_train = df_train.drop(columns=["label"])
y_train = df_train[["label"]]

In [0]:
plt.plot((x_train/255.0).sum(axis=0))
plt.xticks(rotation=45)
plt.show()

In [0]:
plt.plot((x_train/255.0).sum(axis=1))
plt.plot((x_train/255.0).mean(axis=1))

In [0]:
y_train.label.value_counts().plot.bar()

In [0]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)

In [0]:
x_train.shape, y_train.shape

In [0]:
x_train = x_train.values.reshape(42000, 28, 28, 1)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state=42)

In [0]:
from keras.layers import InputLayer, Dense, Activation, ZeroPadding1D, BatchNormalization, Flatten, Conv2D, Input
from keras.layers import AveragePooling2D, MaxPool2D, Dropout
from keras.models import Sequential, Model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras.losses import categorical_crossentropy

In [0]:
#  Tensorboard Colab 
! pip install tensorboardcolab

In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbcolab = TensorBoardColab()

In [0]:
model = Sequential()
model.add(InputLayer(input_shape=(28,28,1)))
model.add(Conv2D(64, (3,3), strides=(1,1), name="Conv2d_1", padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2), name="MaxPool_1"))

model.add(Conv2D(64, (3,3), strides=(1,1), name="Conv2d_2", padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2), name="MaxPool_2"))

model.add(Conv2D(32, (3,3), strides=(1,1), name="Conv2d_3", padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2), name="MaxPool_3"))

model.add(Flatten())
model.add(Dense(64, activation="relu", name="Dense_1"))
model.add(Dropout(0.25))
model.add(Dense(32, activation="relu", name="Dense_2"))
model.add(Dropout(0.25))
model.add(Dense(10, activation="softmax", name="Dense_3"))

In [0]:
"""
In brief, Adagrad performs larger updates for more sparse parameters and smaller updates for less sparse parameter. 
It has good performance with sparse data and training large-scale neural network. 
However, its monotonic learning rate usually proves too aggressive and stops learning too early when training deep neural networks. 
Adadelta is an extension of Adagrad that seeks to reduce its aggressive, monotonically decreasing learning rate. 
RMSprop adjusts the Adagrad method in a very simple way in an attempt to reduce its aggressive, monotonically decreasing learning rate. 
Adam is an update to the RMSProp optimizer which is like RMSprop with momentum.
"""

![Learning Rate Comparision](https://cdn-images-1.medium.com/max/1000/1*OjcTfMw6dmOmP4lRE7Ud-A.jpeg)

In [0]:
# STEP DECAY FUNCTION - https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
# FORMULA =>    lr = lr0 * drop^floor(epoch / epochs_drop)  with SGD

In [0]:
def step_decay(epoch):
  initial_rate= 0.1
  drop = 0.5
  epoch_drop = 3.0
  lrate = initial_rate * math.pow(drop, math.floor((1+ epoch)/epoch_drop))
  return lrate

In [0]:
lrate = LearningRateScheduler(step_decay)

In [0]:
# Writing a custom Callback 
"""
As a digression, a callback is a set of functions to be applied at given stages of the training procedure. 
We can use callbacks to get a view on internal states and statistics of the model during training. 
In our example, we create a custom callback by extending the base class keras.callbacks.
Callback to record loss history and learning rate during the training procedure.
"""

In [0]:
class LossHistory(keras.callbacks.Callback):
  def on_train_begin(self, logs={}):
    self.losses = []
    self.lr = []
  def on_epoch_end(self, batch, logs={}):
    self.losses.append(logs.get("loss"))
    self.lr.append(step_decay(self.losses))

In [0]:
model.compile(optimizer=SGD(), loss=categorical_crossentropy, metrics=["acc"])

In [0]:
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=20, callbacks=[TensorBoardColabCallback(tbcolab)])